# 4 - Medium Level Example - Debugging your Scene with Custom Objects
### Fixed Tilt 2-up with Torque Tube + CLEAN Routine + CustomObject

This journal has examples of various things, some which hav ebeen covered before and some in more depth:

<ul>
    <li> Running a fixed_tilt simulation beginning to end.  </li>
    <li> Creating a 2-up module with torque-tube, and detailed geometry of spacings in xgap, ygap and zgap.  </li>
    <li> Calculating the tracker angle for a specific time, in case you want to use that value to model a fixed_tilt setup.  </li>
    <li> Loading and cleaning results, particularly important when using setups with torquetubes / ygaps.  </li>
    <li> Adding a "Custom Object" or **marker** at the Origin of the Scene, to do a visual sanity-check of the geometry. </li>
</ul>

It will look something like this (without the marker in this visualization):

![What we are trying to re-create](../images_wiki/Journal_example_torquetube.PNG)

### STEPS:

<ol type='1'>
    <li> <a href='#step1'> Specify Working Folder and Import Program </a></li>
    <li> <a href='#step2'> Specify all variables </a></li>
    <li> <a href='#step3'> Create the Radiance Object and generate the Sky </a></li>
    <li> <a href='#step4'> Calculating tracker angle/geometry for a specific timestamp </a></li>
    <li> <a href='#step5'> Making the Module & the Scene,  Visualize and run Analysis </a></li>
    <li> <a href='#step6'> Calculate Bifacial Ratio (clean results) </a></li>
    <li> <a href='#step7'> Add Custom Elements to your Scene Example: Marker at 0,0 position </a></li>
</ol>

<a id='step1'></a>

### 1. Specify Working Folder and Import Program


In [2]:
import os
from pathlib import Path

testfolder = Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP'

print ("Your simulation will be stored in %s" % testfolder)

import bifacial_radiance
import numpy as np

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


<a id='step2'></a>

### 2. Specify all variables for the module and scene

Below find a list of all of the possible parameters for makeModule. 
scene and simulation parameters are also organized below. 
This simulation will be a complete simulation in terms of parameters that you can modify.

The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
This is being run with gendaylit, for one specific timestamp

In [3]:
timestamp = 4020 # Noon, June 17th. 
simulationname = 'Torque_tube_hex_test'

## SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
hub_height = 2.35  # we could also pass clearance_height.   
azimuth_ang=90 # Modules will be facing East.
lat = 37.5
lon = -77.6
nMods = 4   # doing a smaller array for better visualization on this example.
nRows = 2  

# MakeModule Parameters
module_type='my_custom_panel'
x = 1.996      # landscape, sinze x > y. Remember that orientation has been deprecated.
y = 0.991
tilt = 10
numpanels = 2  # doing a 2-up system!
cellLevelModule = False # not doing a cell level module on this example.

# Gaps:
xgap = 0.05  # distance between modules in the row.
ygap = 0.15  # distance between the 2 modules along the collector slope.
zgap = 0.175 # if there is a torquetube, this is the distance between the torquetube and the modules.
# If there is not a module, zgap is the distance between the module and the axis of rotation (relevant for 
# tracking systems. 

# TorqueTube Parameters
torqueTube = True
tubetype = 'Hex'
diameter = 0.15
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY.

<a id='step3'></a>

### 3. Create the Radiance Object and generate the Sky

In [4]:
demo = bifacial_radiance.RadianceObj(simulationname,path = str(testfolder))  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(lat,lon) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(timestamp)  # Noon, June 17th

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Loading albedo, 1 value(s), 0.280 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!
Saving file EPWs\epw_temp.csv, # points: 8760


'skies\\sky2_37.5_-77.33_06_17_13.rad'

<a id='step4'></a>

## 4. Calculating tracker angle/geometry for a specific timestamp

This trick is useful if you are trying to use the fixed-tilt steps in bifacial_radiance to model a tracker for one specific point in time (if you take a picture of a tracker, it looks fixed, right? Well then). 

We assigned a 10 degree tilt at the beginning, but if we were to model a tracker as a fixed-tilt element because we are interested in only one point in time, this routine will tell us what tilt to use. *Please note that to model a tracker as fixed tilt, we suggest passing a hub_height, otherwise you will have to calculate the clearance_height manually.*

<div class="alert alert-warning">
Details: you might have noticed in the previoust tutorial looking at the tracker dictionary, but the way that bifacial_radiance handles tracking: If the tracker is N-S axis azimuth, the surface azimuth of the modules will be set to 90 always, with a tilt that is either positive (for the early morning, facing East), or negative (for the afternoon, facing west).
</div>


In [5]:
# Some tracking parameters that won't be needed after getting this angle:
axis_azimuth = 180
axis_tilt = 0
limit_angle = 60
backtrack = True
tilt = demo.getSingleTimestampTrackerAngle(metdata, timestamp, gcr, axis_azimuth, axis_tilt,limit_angle, backtrack)

print ("\n NEW Calculated Tilt: %s " % tilt)


 NEW Calculated Tilt: -4.67 


<a id='step5'></a>

### 5. Making the Module & the Scene, Visualize and run Analysis

In [6]:
# Making module with all the variables
moduledict=demo.makeModule(name=module_type,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, rewriteModulefile = True, xgap = xgap)

# create a scene with all the variables. 
# Specifying the pitch automatically with the collector width (sceney) returned by moduledict.
# Height has been deprecated as an input. pass clearance_height or hub_height in the scenedict.

sceneDict = {'tilt':tilt,'pitch': np.round(moduledict['sceney'] / gcr,3),
             'hub_height':hub_height,'azimuth':azimuth_ang, 
             'module_type':module_type, 'nMods': nMods, 'nRows': nRows}  

scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict) #makeScene creates a .rad file of the Scene

octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


Module Name: my_custom_panel
REWRITING pre-existing module file. 
Module my_custom_panel successfully created
Created Torque_tube_hex_test.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. It should look like the image at the beginning of the journal. Example:
    
#### rvu -vf views\front.vp -e .01 -pe 0.02 -vp -2 -12 14.5 Torque_tube_hex_test.oct
   
And then proceed happily with your analysis:

In [7]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

sensorsy = 200 # setting this very high to see a detailed profile of the irradiance, including
#the shadow of the torque tube on the rear side of the module.
frontscan, backscan = analysis.moduleAnalysis(scene, modWanted = 2, rowWanted = 1, sensorsy = 200)
frontDict, backDict = analysis.analysis(octfile, demo.name, frontscan, backscan)  # compare the back vs front irradiance  

# print('"Annual" bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
# See comment below of why this line is commented out.

Linescan in process: Torque_tube_hex_test_Front
Linescan in process: Torque_tube_hex_test_Back
Saved: results\irr_Torque_tube_hex_test.csv


<a id='step6'></a>


### 6. Calculate Bifacial Ratio (clean results)

Although we could calculate a bifacial ratio average at this point, this value would be misleading, since some of the sensors generated will fall on the torque tube, the sky, and/or the ground since we have torquetube and ygap in the scene. To calculate the real bifacial ratio average, we must use the clean routines.


In [8]:
resultFile='results/irr_Torque_tube_hex_test.csv'
results_loaded = bifacial_radiance.load.read1Result(resultFile)
print("Printing the dataframe containing the results just calculated in %s: " % resultFile)
results_loaded

Printing the dataframe containing the results just calculated in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\results\irr_Torque_tube_hex_test.csv: 


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,1.033203,6.326542e-17,2.625283,2.535283,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.596467,129.525033,0.214234
1,1.022819,6.262958e-17,2.623119,2.533119,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.599933,129.370167,0.213976
2,1.012435,6.199375e-17,2.620955,2.530955,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.599467,129.216100,0.213721
3,1.002051,6.135792e-17,2.618792,2.528792,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.599067,129.062033,0.213467
4,0.991667,6.072208e-17,2.616628,2.526628,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.598667,128.907933,0.213212
5,0.981283,6.008625e-17,2.614465,2.524465,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.598267,128.753833,0.212957
6,0.970899,5.945042e-17,2.612301,2.522301,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.597900,128.599733,0.212703
7,0.960515,5.881458e-17,2.610137,2.520137,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.597467,128.445633,0.212448
8,0.950131,5.817875e-17,2.607974,2.517974,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.597033,128.291533,0.212193
9,0.939747,5.754292e-17,2.605810,2.515810,a1.0.a0.my_custom_panel.6457,a1.0.a0.my_custom_panel.2310,604.596667,128.137433,0.211938


In [9]:
print("Looking at only 1 sensor in the middle -- position 100 out of the 200 sensors sampled:")
results_loaded.loc[100]

Looking at only 1 sensor in the middle -- position 100 out of the 200 sensors sampled:


x                          -0.00519197
y                         -3.17917e-19
z                              2.40892
rearZ                          2.31892
mattype            a1.0.hextube1c.6457
rearMat                            sky
Wm2Front                       204.946
Wm2Back                        420.946
Back/FrontRatio                2.05392
Name: 100, dtype: object

As an example, we can see above that sensor 100 falls in the hextube, and in the sky. We need to remove this to calculate the real bifacial_gain from the irradiance falling into the modules. To do this we use cleanResult form the load.py module in bifacial_radiance. This finds the invalid materials and sets the irradiance values for those materials to NaN

This might take some time in the current version. 

In [10]:
# Cleaning Results:
# remove invalid materials and sets the irradiance values to NaN
clean_results = bifacial_radiance.load.cleanResult(results_loaded)  

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
print("Sampling the same location as before to see what the results are now:")
clean_results.loc[100]

Sampling the same location as before to see what the results are now:


x                          -0.00519197
y                         -3.17917e-19
z                              2.40892
rearZ                          2.31892
mattype            a1.0.hextube1c.6457
rearMat                            sky
Wm2Front                           NaN
Wm2Back                            NaN
Back/FrontRatio                2.05392
Name: 100, dtype: object

In [12]:
print('CORRECT Annual bifacial ratio average:  %0.3f' %( clean_results['Wm2Back'].sum() / clean_results['Wm2Front'].sum() ))

print ("\n(If we had not done the cleaning routine, the bifacial ratio would have been ", \
      "calculated to %0.3f  <-- THIS VALUE IS WRONG)" %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )) 


CORRECT Annual bifacial ratio average:  0.201

(If we had not done the cleaning routine, the bifacial ratio would have been  calculated to 0.245  <-- THIS VALUE IS WRONG)


<a id='step7'></a>

### 7. Add Custom Elements to your Scene Example: Marker at 0,0 position
This shows how to add a custom element, in this case a Cube, that will be placed in the center of your already created scene to mark the 0,0 location. 

This can be added at any point after makeScene has been run once.  Notice that if this extra element is in the scene and the analysis sensors fall on this element, they will measure irradiance at this element and no the modules.

We are going to create a "MyMarker.rad" file in the objects folder, right after we make the Module. 
This is a prism (so we use 'genbox'), that is black from the ground.rad list of materials ('black')
We are naming it 'CenterMarker'
Its sides are going to be 0.5x0.5x0.5 m 
and We are going to leave its bottom surface coincident with the plane z=0, but going to center on X and Y.

In [13]:
name='MyMarker'
text='! genbox black CenterMarker 0.1 0.1 4 | xform -t -0.05 -0.05 0'
customObject = demo.makeCustomObject(name,text)



Custom Object Name objects\MyMarker.rad


This should have created a MyMarker.rad object on your objects folder.

But creating the object does not automatically adds it to the seen. So let's now add the customObject to the Scene. We are not going to translate it or anything because we want it at the center, but you can pass translation, rotation, and any other XFORM command from Radiance.

I am passing a rotation 0 because xform has to have something (I think) otherwise it gets confused.

In [14]:
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')
# makeOct combines all of the ground, sky and object files into a .oct file.
octfile = demo.makeOct(demo.getfilelist())  


!xform -rz 0 objects\MyMarker.rad
Created Torque_tube_hex_test.oct


appendtoScene appended to the Scene.rad file the name of the custom object we created and the xform transformation we included as text. Then octfile merged this new scene with the ground and sky files.

At this point you should be able to go into a command window (cmd.exe) and check the geometry, and the marker should be there. Example:
    
   #### rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
If you ran the getTrackerAngle detour and appended the marker, it should look like this:

![Marker position at 0,0](..\images_wiki\Journal_example_marker_origin.PNG)

If you do an analysis and any of the sensors hits the Box object we just created, the list of materials in the result.csv file should say something with "CenterMarker" on it. 

#### See more examples of the use of makeCustomObject and appendtoScene on the Bifacial Carport/Canopies Tutorial